# Some OOP for DB and Magic Methods

## 1. Wrapping the whole connection in a context manager
- Out of with CLOSE the cnc ok!

In [88]:
from contextlib import contextmanager
import pyodbc
import sys

@contextmanager
def open_db_cnx(connection_string, commit=False):
    cnx = pyodbc.connect(connection_string)
    cursor = cnx.cursor()
    try:
        yield cursor
    except pyodbc.DatabaseError as err:
        error, = err.args
        sys.stderr.write(error.message)
        cursor.execute("ROLLBACK")
        raise err
    else:
        if commit:
            cursor.execute("COMMIT")
        else:
            cursor.execute("ROLLBACK")
    finally:
        cnx.close()

drv = '{ODBC Driver 17 for SQL Server}'
srv, port, db = '(local)', 1433, 'BikeStores'
usr, pwd = 'user1', 'pass1'
cnxstr = f'DRIVER={drv};SERVER={srv};PORT={port};\
    DATABASE={db};UID={usr};PWD={pwd}'

with open_db_cnx(cnxstr) as cur:
    cur.execute('''SELECT @@Version;''')
    print(cur.fetchone())

print(cnx.closed)

('Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) \n\tSep 24 2019 13:48:23 \n\tCopyright (C) 2019 Microsoft Corporation\n\tDeveloper Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 19045: )\n',)
True


## 2. First OOP aproach

In [37]:
import pyodbc


class MS_DB:
    ''' Doc String'''

    def __init__(self, srv, db, usr, pwd, port=1433):
        self.drv = '{ODBC Driver 17 for SQL Server}'
        self.srv, self.port, self.db = srv, port, db
        self.usr, self.pwd = usr, pwd
        cnxstr = f'DRIVER={self.drv};SERVER={self.srv};PORT={self.port};\
            DATABASE={self.db};UID={self.usr};PWD={self.pwd}'
        print('Cnx Str:', cnxstr)
        self._cnx = pyodbc.connect(cnxstr)
        pyodbc.pooling = False

    def __repr__(self):
        return f'''MS-SQLSrv('{self.usr}', <password hidden>, \
            '{self.srv}', '{self.port}', '{self.db}')'''

    def __str__(self):
        return f"MS-SQLSrv Module for STP on {self.srv}"

    def __del__(self):
        self._cnx.close()
        print("Connection closed.")

    @contextmanager
    def cnx_cur(self, commit :bool = False):
        cur = self._cnx.cursor()
        try:
            yield cur
        except pyodbc.DatabaseError as err:
            error, = err.args
            sys.stderr.write(error.message)
            cur.execute("ROLLBACK")
            raise err
        else:
            if commit:
                cur.execute("COMMIT")
            else:
                cur.execute("ROLLBACK")
        finally:
            cur.close()

try:    
    db = MS_DB('localhost', 'BikeStores', 'user1', 'pass1')
except Exception as e:
    sys.stderr.write(f'Error trying to connect to the db: {e}')
else:
    print('Connection established!')

with db.cnx_cur() as cur:
    qry = ''' SELECT * FROM production.brands
            WHERE brand_name LIKE 's%';'''
    cur.execute(qry)
    print(cur.fetchall())

print(db)


Cnx Str: DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;PORT=1433;            DATABASE=BikeStores;UID=user1;PWD=pass1
Connection closed.
Connection established!
[(6, 'Strider'), (7, 'Sun Bicycles'), (8, 'Surly')]
MS-SQLSrv Module for STP on localhost


## 3. A complete MSSQLdb class

In [96]:
import pyodbc
import pandas as pd


class MSSQL_DB:
    ''' Doc String'''

    def __init__(self, srv, db, usr, pwd, port=1433):
        pyodbc.pooling = False
        self.drv = '{ODBC Driver 17 for SQL Server}'
        self.srv, self.port, self.db = srv, port, db
        self.usr, self.pwd = usr, pwd
        cnxstr = f'DRIVER={self.drv};SERVER={self.srv};PORT={self.port};\
            DATABASE={self.db};UID={self.usr};PWD={self.pwd}'
        print('Cnx Str:', cnxstr)
        self._cnx = pyodbc.connect(cnxstr)
        self._cur = self._cnx.cursor()

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    def __repr__(self):
        return f'''MS-SQLSrv('{self.usr}', <password hidden>, \
            '{self.srv}', '{self.port}', '{self.db}')'''

    def __str__(self):
        return f"MS-SQLSrv Module for STP on {self.srv}"
    
    def commit(self):
        self._cnx.commit()

    def close(self, commit=True):
        if commit:
            self.commit()
        self._cur.close()
        self._cnx.close()
        print('cur & cnx are CLOSED')
    
    def execute(self, sql, params=None):
        self._cur.execute(sql, params or ())

    def fetchall(self):
        return self._cur.fetchall()
    
    def qry_to_df(self, sql, params=None):
        self.execute(sql, params)
        cols = [i[0] for i in self._cur.description]
        dats = [list(xx) for xx in self._cur]
        return pd.DataFrame(data=dats, columns=cols)

In [97]:
with MSSQL_DB('localhost', 'BikeStores', 'user1', 'pass1') as db:
    qry = ''' SELECT * FROM production.brands
            WHERE brand_name LIKE 's%';'''
    # db.execute(qry)
    # print(db.fetchall())
    df = db.qry_to_df(qry)
    display(df)

Cnx Str: DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;PORT=1433;            DATABASE=BikeStores;UID=user1;PWD=pass1


,brand_id,brand_name
0,6,Strider
1,7,Sun Bicycles
2,8,Surly


cur & cnx are CLOSED


In [104]:
try:
    srv, dbnm, usr, pwd = 'localhost', 'BikeStores', 'user1', 'pass1'
    db = MSSQL_DB(srv, dbnm, usr, pwd)
except Exception as e:
    ln = f'''ERROR -> {e}'''
else:
    ln = f'''Conexión con {srv}:{dbnm} establecida'''
finally:
    print(ln)

try:
    qry =''' SELECT * FROM production.brands
        WHERE brand_name LIKE 'a%';'''
    assert 'select' not in qry
    df = db.qry_to_df(qry)
except AssertionError:
    ln = f'''Query NO Permitida'''
except Exception as e:
    ln = f'''ERROR! -> {e}'''
else:
    # create folder
    # write csv
    ln = f'''Query OK, .csv creado!'''
finally:
    display(df)
    print(ln)

try:
    db.close()
except Exception as e:
    ln = 
        
                  

Cnx Str: DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;PORT=1433;            DATABASE=BikeStores;UID=user1;PWD=pass1
Conexión con localhost:BikeStores establecida


,brand_id,brand_name


Query OK, .csv creado!
cur & cnx are CLOSED


## 3b. Trying a complete MSSQLdb class

In [90]:
import pyodbc
import pandas as pd


class MSSQL_DB:
    ''' Doc String'''

    def __init__(self, srv, db, usr, pwd, port=1433):
        pyodbc.pooling = False
        self.drv = '{ODBC Driver 17 for SQL Server}'
        self.srv, self.port, self.db = srv, port, db
        self.usr, self.pwd = usr, pwd
        cnxstr = f'DRIVER={self.drv};SERVER={self.srv};PORT={self.port};\
            DATABASE={self.db};UID={self.usr};PWD={self.pwd}'
        print('Cnx Str:', cnxstr)
        self._cnx = pyodbc.connect(cnxstr)
        self._cur = self._cnx.cursor()

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

    def __repr__(self):
        return f'''MS-SQLSrv('{self.usr}', <password hidden>, \
            '{self.srv}', '{self.port}', '{self.db}')'''

    def __str__(self):
        return f"MS-SQLSrv Module for STP on {self.srv}"
    
    @property
    def cnx(self):
        return self._cnx
    
    @property
    def cur(self):
        return self._cur
    
    def commit(self):
        self.cnx.commit()

    def close(self, commit=True):
        if commit:
            self.commit()
        self.cur.close()
        self.cnx.close()
        print('cur & cnx are CLOSED')
    
    def execute(self, sql, params=None):
        self.cur.execute(sql, params or ())

    def fetchall(self):
        return self.cur.fetchall()
    
    def qry_to_df(self, sql, params=None):
        self.execute(sql, params)
        cols = [i[0] for i in self.cur.description]
        dats = [list(xx) for xx in self.cur]
        return pd.DataFrame(data=dats, columns=cols)

In [92]:
with MSSQL_DB('localhost', 'BikeStores', 'user1', 'pass1') as db:
    qry = ''' SELECT * FROM production.brands
            WHERE brand_name LIKE 's%';'''
    # db.execute(qry)
    # print(db.fetchall())
    df = db.qry_to_df(qry)
    display(df)

Cnx Str: DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;PORT=1433;            DATABASE=BikeStores;UID=user1;PWD=pass1


,brand_id,brand_name
0,6,Strider
1,7,Sun Bicycles
2,8,Surly


cur & cnx are CLOSED


## 4. Use with to manage the connection
- Out of with DO NOT close the cnx.

In [ ]:
import pyodbc

drv = '{ODBC Driver 17 for SQL Server}'
srv, port, db = '(local)', 1433, 'BikeStores'
usr, pwd = 'user1', 'pass1'
cnxstr = f'DRIVER={drv};SERVER={srv};PORT={port};\
    DATABASE={db};UID={usr};PWD={pwd}'

try:
    cnx = pyodbc.connect(cnxstr)
except Exception as e:
    print(f'''Error! {e}''')
else:
    print(f'''Cnx to {srv}:{db} Established!''')

with cnx:
    cur = cnx.cursor()
    cur.execute('''SELECT @@Version;''')
    print(cur.fetchone())

cur.close()
cnx.close()
print(cnx.closed)

## 1. Some MM

In [ ]:
class MyClass:
    def __init__(self, name) -> None:
        self.name = name
        print('__init__')
    def __enter__(self):
        print('__enter__')
        return self
    def __exit__(self, ex_type, ex_value, ex_tb):
        print("__exit__")
    def __del__(self):
        print("__del__")
    def __str__(self):
        return self.name
    def __repr__(self) -> str:
        return f'''{__class__.__name__}('{self.name}')'''

o1 = MyClass('o1')   
print(o1)
print(o1.__class__.__name__)
print(str(o1))
print(repr(o1))
print()

with MyClass('obj1') as o:
    print(f'''I'm {str(o)}, an instance {repr(o)}''')

## 2. LOGs

In [ ]:
import datetime as dtm

fstamp = dtm.datetime.now().strftime('%Y%m%d%H%M')

class Log:
    def __init__(self, fn):
        self.fn = fn
        self.fp = None
        self.closed = None
    def logging(self, msg):
        tstamp = dtm.datetime.now().strftime('%b %d %H:%M')
        self.fp.write(f'''{tstamp} {msg}\n''')
    def __enter__(self):
        print('__enter__ Mm')
        self.closed = False
        self.fp = open(self.fn, 'a+')
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("__exit__")
        self.closed = True
        self.fp.close()

with Log(f'''logs_files/log_{fstamp}''') as lf:
    print('Main')
    for msg in 't1', 'TEST#2', 3, dtm.datetime.now():
        lf.logging(msg)
        print(f'''lf status: {lf.closed}''')

print(f'''lf status: {lf.closed}''')
        

## 91. Decorators 1

In [ ]:
def null_decorator(func):
    return func
# A callable (null_decorator) that takes a callable as input (func), and return
# another callable (func). The input is the parameter of the decorator

# first use of this 'null_decorator':
@null_decorator
def greet():
    return 'Hello'

#greet = null_decorator(greet)   # This is what @null_decorator do over greet funct

greet()

##  92. Decorators 2
- Wrapper funct. add code to the paseed funct. as argument
- Decorator returns the wrapper funct. (actually the funct. passed wrappered)

In [46]:
def deco_wrapper_func(fun):
    def wrapper_func():
        print(f'''wrapper code before '{fun}' ''')
        fun()
        print(f'''wrapper code after '{fun}' ''' )
    return wrapper_func

def say_slogan(group='group'):
    print(f'''An remember {group} 'you want it you have it' ''')

dec_say_slgn = deco_wrapper_func(say_slogan)

## primitive func
say_slogan()

## linear decorated func
dec_say_slgn()

An remember group 'you want it you have it' 
wrapper code before '<function say_slogan at 0x000002892AC48D60>' 
An remember group 'you want it you have it' 
wrapper code after '<function say_slogan at 0x000002892AC48D60>' 


>If I use dcorator suggar sintax y lost primitive funct.

In [48]:
@deco_wrapper_func
def say_slgn_2(group='group'):
     print(f'''An remember {group} 'you ask for it you get it' ''')

say_slgn_2()

wrapper code before '<function say_slgn_2 at 0x000002892AC485E0>' 
An remember group 'you ask for it you get it' 
wrapper code after '<function say_slgn_2 at 0x000002892AC485E0>' 


## 93. Example using @property
- In Python, property() is a built-in function that creates and returns a property object.
- A property object has three methods, getter(), setter(), and delete().
- property func convert methods in attributes to ensure encapsulation¿?

In [56]:
## Classic Class that store temperature in Celsius degrees
class Celsius:
    def __init__(self, temperature = 0):
        self.set_temperature(temperature)

    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32

    def get_temperature(self):
        return self._temperature

    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        self._temperature = value

t1 = Celsius(16)
print(t1.to_fahrenheit())
print(t1.get_temperature())
t1.set_temperature(t1.get_temperature() + 3)
print(t1.__dict__)
#dir(t1)
#print(Celsius.__dict__)

Celsius
60.8
16
{'_temperature': 19}


In [67]:
## Refactor code using property in order to avoid using Celsius Class methods
class Celsius:
    def __init__(self, temperature = 0):
        self.set_temperature(temperature)
    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32
    def get_temperature(self):
        return self._temperature
    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        self._temperature = value
    def del_temperature(self):
        print('del_self-temp')
        del self._temperature
    temperature = property(get_temperature, set_temperature, del_temperature,
                           '''I'm the 'temperature' property of Celsius''')
    # # make empty property
    # temperature = property()
    # # assign fget
    # temperature = temperature.getter(get_temperature)
    # # assign fset
    # temperature = temperature.setter(set_temperature)

t1 = Celsius(16)
print(t1.to_fahrenheit())
print(t1.temperature)
t1.temperature = t1.temperature + 3
print(t1.temperature)
#print(temperature)
print(t1.__dict__)
#dir(t1)
#print(Celsius.__dict__)

60.8
16
19
{'_temperature': 19}


In [69]:
## The pythonic way to deal with the above problem is to use @property.
class Celsius:
    def __init__(self, temperature = 0):
        self._temperature = temperature
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32
    @property
    def temperature(self):
        '''I'm the 'temperature' property of Celsius'''
        print("Getting value")
        return self._temperature
    @temperature.setter
    def temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value
    @temperature.deleter
    def temperature(self):
        print('del_self-temp')
        del self._temperature

t1 = Celsius(12)
print(t1.temperature)
t1.temperature = t1.temperature + 5
print(t1.temperature)

# https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python
# https://www.machinelearningplus.com/python/python-property/

Getting value
12
Getting value
Setting value
Getting value
17


- https://stackoverflow.com/questions/3783238/python-database-connection-close
- https://stackoverflow.com/questions/38076220/python-mysqldb-connection-in-a-class/38078544#38078544
- https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python

## 1. Some OOP 'magic methods'


In [ ]:
class A:
    def __init__(self, objnm):
        self.jm0, self.objnm = 'jm0', objnm

    def __str__(self):
        return f'''I'm {self.objnm} Object from 'a' class'''
    
    def __del__(self):
        print(f'''Object {self.objnm} leaving the game''')

    